In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tf.core.files import expanduser as ex, chDir, getCwd, fileMake, fileExists
from tf.core.helpers import console
from tf.lib import writeSets
from tf.app import use

from helpers import SQL, VERSIONS

# Weed out the user data

We work with an mysqldump of the databases `shebanq_web` and `shebanq_note`.

## Preparation

Make a mysqldump of the real shebanq at ancient-data.org

```
k
kset shebanq
ksh
```

Now you are in a shell in the container where shebanq is being served on k8s.

```
cd src/scripts
./backup.sh ALL
exit
```

Now the mysqldumps are in the `/app/backup` directory, which is on a PVC.

Back in the shell on your own computer do

```
cd ~/github/ETCBC/shebanq-local
kfromapp backup
```

Now the backup directory has been copied from k8s to your local computer, in `app/backup`.
From now on we work on the local computer.

Start the local shebanq in production mode:

```
./shebanq.sh up
```

In another shell, still in directory `~/github/ETCBC/shebanq-local` do

```
./shebanq.sh sh
```

Now you are in a shell of the local shebanq, from where you can retore the backup we just
fetched from the production shebanq:

```
cd src/scripts
./restore.sh ALL
```

Now we export the data as sql definition files plus tsv data files, by means of mysqldump with the `--tab` option.

In the same shell where you did the restore, do

```
./export.sh
```

Now there are two directories added to the `app/backup` folder, namely `shebanq_web` and `shebanq_note`, and they
contain the definition and data files for each table in that database.

Open yet another shell in the same directory on your local computer.
Now you can explore the database by

```
./shebanq.sh browse admin
```

When asked for a password, it is `wajehior`.

## The weeding itself

Now we are in a position to weed out data from the exported files, after which we can re-import these files
by means of an import script.

If something goes wrong, we can always go back to the original backup by means of the restore script:

```
app sh
./restore.sh ALL
exit
```

We start by reading the tables into rows of fields in Python

In [3]:
BASEDIR = ex("~/github/ETCBC/shebanq-local")

S = SQL(f"{BASEDIR}/backup", zapTables={"web2py_session_shebanq", "auth_event", "auth_cas"})
S.stats()

Database shebanq_note:
	Table note                     :   222517 rows
Database shebanq_web:
	Table auth_group               :      175 rows
	Table auth_membership          :      175 rows
	Table auth_permission          :        0 rows
	Table auth_user                :      175 rows
	Table monads                   :   701052 rows
	Table organization             :       75 rows
	Table project                  :      112 rows
	Table query                    :     1130 rows
	Table query_exe                :     1157 rows
	Table uploaders                :        1 rows


### Queries en notes

We remove all queries that are not published, and we remove all notes that are not shared.

Queries: weed out unpublished query executions:

In [8]:
S.keep("shebanq_web", "query_exe", lambda r: r[8] == "T")
S.stats()

Database shebanq_note:
	Table note                     :   222527 rows
Database shebanq_web:
	Table auth_group               :     1673 rows
	Table auth_membership          :     1668 rows
	Table auth_permission          :        0 rows
	Table auth_user                :     1669 rows
	Table monads                   :  6295774 rows
	Table organization             :      526 rows
	Table project                  :      782 rows
	Table query                    :     7095 rows
	Table query_exe                :     1157 rows
	Table uploaders                :        1 rows


Notes: weed out notes that are neither published nor shared:

In [9]:
S.keep("shebanq_note", "note", lambda r: r[9] == "T" or r[11] == "T")
S.stats()

Database shebanq_note:
	Table note                     :   222517 rows
Database shebanq_web:
	Table auth_group               :     1673 rows
	Table auth_membership          :     1668 rows
	Table auth_permission          :        0 rows
	Table auth_user                :     1669 rows
	Table monads                   :  6295774 rows
	Table organization             :      526 rows
	Table project                  :      782 rows
	Table query                    :     7095 rows
	Table query_exe                :     1157 rows
	Table uploaders                :        1 rows


Now we trim the database of all data that is related to the removed rows and is no longer needed.

We weed the `monads` detail records. 

In [10]:
S.trimDetails("shebanq_web", "query_exe", "shebanq_web", "monads", 0)
S.stats()

Database shebanq_note:
	Table note                     :   222517 rows
Database shebanq_web:
	Table auth_group               :     1673 rows
	Table auth_membership          :     1668 rows
	Table auth_permission          :        0 rows
	Table auth_user                :     1669 rows
	Table monads                   :   701052 rows
	Table organization             :      526 rows
	Table project                  :      782 rows
	Table query                    :     7095 rows
	Table query_exe                :     1157 rows
	Table uploaders                :        1 rows


We weed the `query` master records.

In [11]:
S.trimMaster("shebanq_web", "query_exe", 10, "shebanq_web", "query")
S.stats()

Database shebanq_note:
	Table note                     :   222517 rows
Database shebanq_web:
	Table auth_group               :     1673 rows
	Table auth_membership          :     1668 rows
	Table auth_permission          :        0 rows
	Table auth_user                :     1669 rows
	Table monads                   :   701052 rows
	Table organization             :      526 rows
	Table project                  :      782 rows
	Table query                    :     1130 rows
	Table query_exe                :     1157 rows
	Table uploaders                :        1 rows


Remove the superfluous projects and organizations.

In [12]:
S.trimMaster("shebanq_web", "query", 9, "shebanq_web", "organization")
S.stats()

Database shebanq_note:
	Table note                     :   222517 rows
Database shebanq_web:
	Table auth_group               :     1673 rows
	Table auth_membership          :     1668 rows
	Table auth_permission          :        0 rows
	Table auth_user                :     1669 rows
	Table monads                   :   701052 rows
	Table organization             :       75 rows
	Table project                  :      782 rows
	Table query                    :     1130 rows
	Table query_exe                :     1157 rows
	Table uploaders                :        1 rows


In [13]:
S.trimMaster("shebanq_web", "query", 8, "shebanq_web", "project")
S.stats()

Database shebanq_note:
	Table note                     :   222517 rows
Database shebanq_web:
	Table auth_group               :     1673 rows
	Table auth_membership          :     1668 rows
	Table auth_permission          :        0 rows
	Table auth_user                :     1669 rows
	Table monads                   :   701052 rows
	Table organization             :       75 rows
	Table project                  :      112 rows
	Table query                    :     1130 rows
	Table query_exe                :     1157 rows
	Table uploaders                :        1 rows


We collect the users that are still connected to the current queries and notes.

In [14]:
userIds = S.getIds("shebanq_web", "query", 4) | S.getIds("shebanq_note", "note", 6) | S.getIds("shebanq_web", "uploaders", 0)

In [15]:
S.trimTable("shebanq_web", "auth_user", 0, userIds)
S.stats()

Database shebanq_note:
	Table note                     :   222517 rows
Database shebanq_web:
	Table auth_group               :     1673 rows
	Table auth_membership          :     1668 rows
	Table auth_permission          :        0 rows
	Table auth_user                :      175 rows
	Table monads                   :   701052 rows
	Table organization             :       75 rows
	Table project                  :      112 rows
	Table query                    :     1130 rows
	Table query_exe                :     1157 rows
	Table uploaders                :        1 rows


In [16]:
S.trimTable("shebanq_web", "auth_membership", 1, userIds)
S.stats()

Database shebanq_note:
	Table note                     :   222517 rows
Database shebanq_web:
	Table auth_group               :     1673 rows
	Table auth_membership          :      175 rows
	Table auth_permission          :        0 rows
	Table auth_user                :      175 rows
	Table monads                   :   701052 rows
	Table organization             :       75 rows
	Table project                  :      112 rows
	Table query                    :     1130 rows
	Table query_exe                :     1157 rows
	Table uploaders                :        1 rows


In [17]:
S.trimMaster("shebanq_web", "auth_membership", 1, "shebanq_web", "auth_group")
S.stats()

Database shebanq_note:
	Table note                     :   222517 rows
Database shebanq_web:
	Table auth_group               :      175 rows
	Table auth_membership          :      175 rows
	Table auth_permission          :        0 rows
	Table auth_user                :      175 rows
	Table monads                   :   701052 rows
	Table organization             :       75 rows
	Table project                  :      112 rows
	Table query                    :     1130 rows
	Table query_exe                :     1157 rows
	Table uploaders                :        1 rows


Strip data from remaining users.

In [19]:
S.zapFields("shebanq_web", "auth_user", 3, 4, 5, 6, 7)

Database shebanq_note:
	Table note                     :   222517 rows
Database shebanq_web:
	Table auth_group               :      175 rows
	Table auth_membership          :      175 rows
	Table auth_permission          :        0 rows
	Table auth_user                :      175 rows
	Table monads                   :   701052 rows
	Table organization             :       75 rows
	Table project                  :      112 rows
	Table query                    :     1130 rows
	Table query_exe                :     1157 rows
	Table uploaders                :        1 rows


In [20]:
S.data["shebanq_web"]["auth_user"][0:10]

[['2', 'Dirk', 'Roorda', '\\N', '\\N', '\\N', '\\N', '\\N'],
 ['4', 'Oliver', 'Glanz', '\\N', '\\N', '\\N', '\\N', '\\N'],
 ['11', 'Willem', 'van Peursen', '\\N', '\\N', '\\N', '\\N', '\\N'],
 ['13', 'Gino', 'Kalkman', '\\N', '\\N', '\\N', '\\N', '\\N'],
 ['15', 'Reinoud', 'Oosting', '\\N', '\\N', '\\N', '\\N', '\\N'],
 ['16', 'Martijn', 'Naaijer', '\\N', '\\N', '\\N', '\\N', '\\N'],
 ['19', 'Christiaan', 'Erwich', '\\N', '\\N', '\\N', '\\N', '\\N'],
 ['20', 'Joep', 'Dubbink', '\\N', '\\N', '\\N', '\\N', '\\N'],
 ['67', 'Femke', 'Siebesma', '\\N', '\\N', '\\N', '\\N', '\\N'],
 ['84', 'Nicolai', 'Winther-Nielsen', '\\N', '\\N', '\\N', '\\N', '\\N']]

Write the weeded data back to the files:

In [22]:
S.writeData()

## Re-import

When the weeding is done, import the new data as follows:

```
app sh
./import.sh
```

## Backup

If all is well, make a backup dump and put it in the content folder.

still in the same shell as above:

```
./backup.sh
exit
```

And then in the local shell:

```
mv backup/shebanq_web.sql.gz backup/shebanq_note.sql.gz content
```


# Check monad sequence

In [23]:
VNEXT = {
    "4": "4b",
    "4b": "2016",
    "2016": "2017",
    "2017": "2021",
    "c": "2021",
}

We need to unzip the bzipped data files. 
We unzip them to the downloads folder, if they are not already there.

We take the zipped data files from the BHSA repo.

In [15]:
MQL_DIR = ex("~/Downloads")

In [17]:
%%bash -s {BASEDIR}

for version in 4 4b 2016 2017 c 2021
do
    mqlzipfile=$1/../bhsa/shebanq/${version}/shebanq_etcbc${version}.mql.bz2
    mqlfile=~/Downloads/shebanq_etcbc${version}.mql

    if [[ ! -f "$mqlfile" ]]; then
        echo "unzipping $mqlzipfile"
        bunzip2 -k -c "$mqlzipfile" > "$mqlfile"
    fi
done

unzipping /Users/me/github/ETCBC/shebanq-local/../bhsa/shebanq/2021/shebanq_etcbc2021.mql.bz2


In [18]:
def checkSeq(version):
    mqlFile = f"{MQL_DIR}/shebanq_etcbc{version}.mql"

    console(f"Checking {version} from {mqlFile}")
    
    curMonad = 0
    gaps = []
    
    with open(mqlFile) as fh:

        skip = True
        
        for (ln, line) in enumerate(fh):
            if line == "WITH OBJECT TYPE[word]\n":
                skip = False

            if line == "GO\n":
                skip = True

            if skip:
                continue

            if line.startswith("FROM MONADS"):
                monad = int(line.split("=", 1)[1].replace("{", "").replace("}", "").strip())

                if curMonad + 1 != monad:
                    gaps.append((ln + 1, curMonad, monad))
                    
                curMonad = monad

    nGaps = len(gaps)
    
    console(f"\tlast monad = {monad}")
    console(f"\tthere were {nGaps} gaps", error=nGaps > 0)

    for (ln, b, e) in gaps:
        console(f"\t\tline {ln}: gap from {b} to {e}", error=True)

    
                
    

In [19]:
for v in VERSIONS:
    checkSeq(v)

Checking 4 from ~/Downloads/shebanq_etcbc4.mql
	last monad = 426555
	there were 0 gaps
Checking 4b from ~/Downloads/shebanq_etcbc4b.mql
	last monad = 426568
	there were 0 gaps
Checking 2016 from ~/Downloads/shebanq_etcbc2016.mql
	last monad = 426581
	there were 0 gaps
Checking 2017 from ~/Downloads/shebanq_etcbc2017.mql
	last monad = 426584
	there were 0 gaps
Checking c from ~/Downloads/shebanq_etcbcc.mql
	last monad = 426584
	there were 0 gaps
Checking 2021 from ~/Downloads/shebanq_etcbc2021.mql
	last monad = 426590
	there were 0 gaps


# Map slots of all non-2021 versions to slots of 2021

In [20]:
A = {}

During the execution of the following cell you'll see some errors in the 4, 4b versions having to do with the feature `voc_lex_utf8`.
This is not a problem.

In [21]:
for v in VERSIONS:
    A[v] = use(f"ETCBC/bhsa:clone", checkout="clone", mod=[], version=v, source="~/repo-archive/github")

**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
book,39,10937.31,100
chapter,929,459.16,100
lex,9225,46.24,100
verse,23213,18.38,100
half_verse,44682,9.55,100
sentence,66045,6.46,100
sentence_atom,66701,6.40,100
clause,87978,4.85,100
clause_atom,90144,4.73,100
phrase,254664,1.67,100


App config error(s) in lex:
	template: feature voc_lex_utf8 not loaded
	label: feature voc_lex_utf8 not loaded


**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
book,39,10937.64,100
chapter,929,459.17,100
lex,9238,46.18,100
verse,23213,18.38,100
half_verse,45180,9.44,100
sentence,63586,6.71,100
sentence_atom,64354,6.63,100
clause,88011,4.85,100
clause_atom,90554,4.71,100
phrase,253161,1.68,100


...

Name,# of nodes,# slots / node,% coverage
book,39,10938.05,100
chapter,929,459.19,100
lex,9233,46.20,100
verse,23213,18.38,100
half_verse,45180,9.44,100
sentence,63711,6.70,100
sentence_atom,64486,6.62,100
clause,88101,4.84,100
clause_atom,90669,4.70,100
phrase,253187,1.68,100


**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
book,39,10938.05,100
chapter,929,459.19,100
lex,9233,46.20,100
verse,23213,18.38,100
half_verse,45180,9.44,100
sentence,63727,6.69,100
sentence_atom,64525,6.61,100
clause,88121,4.84,100
clause_atom,90688,4.70,100
phrase,253207,1.68,100


**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
book,39,10938.21,100
chapter,929,459.19,100
lex,9230,46.22,100
verse,23213,18.38,100
half_verse,45179,9.44,100
sentence,63717,6.70,100
sentence_atom,64514,6.61,100
clause,88131,4.84,100
clause_atom,90704,4.70,100
phrase,253203,1.68,100


In [24]:
MAPPINGS_FROM = {}
MAPPINGS_GAPS = {}

for v in reversed(VERSIONS):
    if v == "2021":
        continue

    console(f"map {v}-slots to 2021 slots ...")
    nextV = VNEXT[v]

    mapFeat = f"omap@{v}-{nextV}"
    A[nextV].load(mapFeat)
    smap = A[nextV].api.Es(mapFeat).f
    maxSlot = A[v].api.F.otype.maxSlot

    thisMapping = {}
    theseGaps = {}

    for n in range(1, maxSlot + 1):
        x = smap(n)

        if x:
            thisMapping[n] = list(x)[0][0]
        else:
            theseGaps[n] = (v, None)
        
    if nextV == "2021":
        MAPPINGS_FROM[v] = thisMapping
    else:
        remainingMapping = MAPPINGS_FROM[nextV]
        MAPPINGS_FROM[v] = {}
        fullMapping = MAPPINGS_FROM[v]
        
        for n in range(1, maxSlot + 1):
            if n in thisMapping:
                nn = thisMapping[n]

                if nn in remainingMapping:
                    fullMapping[n] = remainingMapping[nn]
                else:
                    theseGaps[n] = (nextV, nn)
                    
    MAPPINGS_GAPS[v] = theseGaps
    nGaps = len(theseGaps)
    console(f"\t{nGaps} gaps")

map c-slots to 2021 slots ...
	0 gaps
map 2017-slots to 2021 slots ...
	0 gaps
map 2016-slots to 2021 slots ...
	0 gaps
map 4b-slots to 2021 slots ...
	0 gaps
map 4-slots to 2021 slots ...
	0 gaps


# Extra check on gaps

In [25]:
for v in VERSIONS:
    if v == "2021":
        continue
        
    maxSlot = A[v].api.F.otype.maxSlot

    gaps = 0
    thisMapping = MAPPINGS_FROM[v]
    
    for n in range(1, maxSlot):
        if n not in thisMapping:
            gaps += 1

    console(f"mapping {v} to 2021 has {gaps} gaps")

mapping 4 to 2021 has 0 gaps
mapping 4b to 2021 has 0 gaps
mapping 2016 to 2021 has 0 gaps
mapping 2017 to 2021 has 0 gaps
mapping c to 2021 has 0 gaps


# Some values

In [26]:
for n in (100000, 200000, 300000, 400000):
    for v in reversed(VERSIONS):
        if v == "2021":
            continue
    
        console(f"{v:>4}-slot {n} maps to {MAPPINGS_FROM[v][n]}")
    console("")

   c-slot 100000 maps to 100001
2017-slot 100000 maps to 100001
2016-slot 100000 maps to 100002
  4b-slot 100000 maps to 100008
   4-slot 100000 maps to 100010

   c-slot 200000 maps to 200003
2017-slot 200000 maps to 200003
2016-slot 200000 maps to 200004
  4b-slot 200000 maps to 200014
   4-slot 200000 maps to 200021

   c-slot 300000 maps to 300003
2017-slot 300000 maps to 300003
2016-slot 300000 maps to 300004
  4b-slot 300000 maps to 300014
   4-slot 300000 maps to 300022

   c-slot 400000 maps to 400004
2017-slot 400000 maps to 400004
2016-slot 400000 maps to 400007
  4b-slot 400000 maps to 400016
   4-slot 400000 maps to 400027



# Turn the query results into a TF set

We save the query result of all published queries as a TF set in the contents directory.

In [27]:
S.writeQResultsTF(MAPPINGS_FROM, f"{BASEDIR}/content")

# Write documentation pages for the queries

In [59]:
S.genQueryPages(f"{BASEDIR}/docs")

Cleaning previous results ... 
Gathering projects ... 
Gathering organizations ... 
Gathering users ... 
Gathering queries ... 
Gathering query executions ... 
Generating pages ... 
Generated 1157 pages for 1130 queries
